# 基于 Chat Completions API 实现外部函数调用

**2023年6月20日，OpenAI 官方在 Chat Completions API 原有的三种不同角色设定（System, Assistant, User）基础上，新增了 Function Calling 功能。**

![function_calling_openai_blog](images/function_calling_openai_blog.png)

**[详见OpenAI Blog](https://openai.com/blog/function-calling-and-other-api-updates)**


`functions` 是 Chat Completion API 中的可选参数，用于提供函数定义。其目的是使 GPT 模型能够生成符合所提供定义的函数参数。请注意，API不会实际执行任何函数调用。开发人员需要使用GPT 模型输出来执行函数调用。

如果提供了`functions`参数，默认情况下，GPT 模型将决定在何时适当地使用其中一个函数。

可以通过将`function_call`参数设置为`{"name": "<insert-function-name>"}`来强制 API 使用指定函数。

同时，也支持通过将`function_call`参数设置为`"none"`来强制API不使用任何函数。

如果使用了某个函数，则响应中的输出将包含`"finish_reason": "function_call"`，以及一个具有该函数名称和生成的函数参数的`function_call`对象。


![function_calling](images/function_calling.png)


## 概述

本 Notebook 介绍了如何将 Chat Completions API 与外部函数结合使用，以扩展 GPT 模型的功能。包含以下2个部分：
- 如何使用 `functions` 参数
- 如何使用 `function_call` 参数
- 使用 GPT 模型生成函数和参数
- 实际执行 GPT 模型生成的函数（以 SQL 查询为例）

### 注意：本示例直接构造 HTTP 请求访问 OpenAI API，因此无需使用 openai Python SDK。

## 安装依赖包

In [ ]:
!pip install scipy tenacity tiktoken termcolor openai requests

In [1]:
import json
import requests
import os
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-3.5-turbo"

### 定义工具函数

首先，让我们定义一些用于调用聊天完成 API 的实用工具，并维护和跟踪对话状态。

In [61]:
# 使用了retry库，指定在请求失败时的重试策略。
# 这里设定的是指数等待（wait_random_exponential），时间间隔的最大值为40秒，并且最多重试3次（stop_after_attempt(3)）。
# 定义一个函数chat_completion_request，主要用于发送 聊天补全 请求到OpenAI服务器
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):

    # 设定请求的header信息，包括 API_KEY
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + os.getenv("OPENAI_API_KEY"),
    }

    # 设定请求的JSON数据，包括GPT 模型名和要进行补全的消息
    json_data = {"model": model, "messages": messages}

    # 如果传入了functions，将其加入到json_data中
    if functions is not None:
        json_data.update({"functions": functions})

    # 如果传入了function_call，将其加入到json_data中
    if function_call is not None:
        json_data.update({"function_call": function_call})

    # 尝试发送POST请求到OpenAI服务器的chat/completions接口
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        # 返回服务器的响应
        return response

    # 如果发送请求或处理响应时出现异常，打印异常信息并返回
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [62]:
# 定义一个函数pretty_print_conversation，用于打印消息对话内容
def pretty_print_conversation(messages):

    # 为不同角色设置不同的颜色
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }

    # 遍历消息列表
    for message in messages:

        # 如果消息的角色是"system"，则用红色打印“content”
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"user"，则用绿色打印“content”
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"assistant"，并且消息中包含"function_call"，则用蓝色打印"function_call"
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant[function_call]: {message['function_call']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"assistant"，但是消息中不包含"function_call"，则用蓝色打印“content”
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant[content]: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"function"，则用品红色打印“function”
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


### 如何使用 functions 参数

这段代码定义了两个可以在程序中调用的函数，分别是获取当前天气和获取未来N天的天气预报。

每个函数(function)都有其名称、描述和需要的参数（包括参数的类型、描述等信息）。

![functions_param](images/functions_param.png)

我们将把这些传递给 Chat Completions API，以生成符合规范的函数。

In [63]:
# 定义一个名为functions的列表，其中包含两个字典，这两个字典分别定义了两个功能的相关参数

# 第一个字典定义了一个名为"get_current_weather"的功能
functions = [
    {
        "name": "get_current_weather",  # 功能的名称
        "description": "Get the current weather",  # 功能的描述
        "parameters": {  # 定义该功能需要的参数
            "type": "object",
            "properties": {  # 参数的属性
                "location": {  # 地点参数
                    "type": "string",  # 参数类型为字符串
                    "description": "The city and state, e.g. San Francisco, CA",  # 参数的描述
                },
                "format": {  # 温度单位参数
                    "type": "string",  # 参数类型为字符串
                    "enum": ["celsius", "fahrenheit"],  # 参数的取值范围
                    "description": "The temperature unit to use. Infer this from the users location.",  # 参数的描述
                },
            },
            "required": ["location", "format"],  # 该功能需要的必要参数
        },
    },
    # 第二个字典定义了一个名为"get_n_day_weather_forecast"的功能
    {
        "name": "get_n_day_weather_forecast",  # 功能的名称
        "description": "Get an N-day weather forecast",  # 功能的描述
        "parameters": {  # 定义该功能需要的参数
            "type": "object",
            "properties": {  # 参数的属性
                "location": {  # 地点参数
                    "type": "string",  # 参数类型为字符串
                    "description": "The city and state, e.g. San Francisco, CA",  # 参数的描述
                },
                "format": {  # 温度单位参数
                    "type": "string",  # 参数类型为字符串
                    "enum": ["celsius", "fahrenheit"],  # 参数的取值范围
                    "description": "The temperature unit to use. Infer this from the users location.",  # 参数的描述
                },
                "num_days": {  # 预测天数参数
                    "type": "integer",  # 参数类型为整数
                    "description": "The number of days to forecast",  # 参数的描述
                }
            },
            "required": ["location", "format", "num_days"]  # 该功能需要的必要参数
        },
    },
]


这段代码首先定义了一个`messages`列表用来存储聊天的消息，然后向列表中添加了系统和用户的消息。

然后，它使用了之前定义的`chat_completion_request`函数发送一个请求，传入的参数包括消息列表和函数列表。

在接收到响应后，它从JSON响应中解析出助手的消息，并将其添加到消息列表中。

最后，它打印出 GPT 模型回复的消息。

**（如果我们询问当前天气，GPT 模型会回复让你给出更准确的问题。）**

In [64]:
# 定义一个空列表messages，用于存储聊天的内容
messages = []

# 使用append方法向messages列表添加一条系统角色的消息
messages.append({
    "role": "system",  # 消息的角色是"system"
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."  # 消息的内容
})

# 向messages列表添加一条用户角色的消息
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "What's the weather like today"  # 用户询问今天的天气情况
})

# 使用定义的chat_completion_request函数发起一个请求，传入messages和functions作为参数
chat_response = chat_completion_request(
    messages, functions=functions
)

# 解析返回的JSON数据，获取助手的回复消息
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的回复消息添加到messages列表中
messages.append(assistant_message)

pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

assistant[content]: Sure, I can help with that. Could you please provide me with the name of the city and state you are interested in knowing the weather for?



**(我们需要提供更详细的信息，以便于 GPT 模型为我们生成适当的函数和对应参数。)**

In [7]:
type(assistant_message)

dict

In [8]:
assistant_message

{'role': 'assistant',
 'content': 'Sure, I can help with that. May I know your current location so I can provide you with the weather information?',
 'refusal': None}

## 使用 GPT 模型生成函数和对应参数

下面这段代码先向messages列表中添加了用户的位置信息。

然后再次使用了chat_completion_request函数发起请求，只是这次传入的消息列表已经包括了用户的新消息。

在获取到响应后，它同样从JSON响应中解析出助手的消息，并将其添加到消息列表中。

最后，打印出助手的新的回复消息。

In [65]:
# 向messages列表添加一条用户角色的消息，用户告知他们在苏格兰的格拉斯哥
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "I'm in Shanghai, China."  # 用户的消息内容
})

# 再次使用定义的chat_completion_request函数发起一个请求，传入更新后的messages和functions作为参数
chat_response = chat_completion_request(
    messages, functions=functions
)

# 解析返回的JSON数据，获取助手的新的回复消息
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的新的回复消息添加到messages列表中
messages.append(assistant_message)

pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

assistant[content]: Sure, I can help with that. Could you please provide me with the name of the city and state you are interested in knowing the weather for?

user: I'm in Shanghai, China.

assistant[function_call]: {'name': 'get_current_weather', 'arguments': '{"location":"Shanghai, China","format":"celsius"}'}



In [10]:
chat_response.json()

{'id': 'chatcmpl-AmbpfVsvNs6cYSZyrq9CEfjqFfhW3',
 'object': 'chat.completion',
 'created': 1736148535,
 'model': 'gpt-3.5-turbo-0125',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'function_call': {'name': 'get_current_weather',
     'arguments': '{"location":"Shanghai, China","format":"celsius"}'},
    'refusal': None},
   'logprobs': None,
   'finish_reason': 'function_call'}],
 'usage': {'prompt_tokens': 230,
  'completion_tokens': 24,
  'total_tokens': 254,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 0,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'system_fingerprint': None}

这段代码的逻辑大体与上一段代码相同，区别在于这次用户的询问中涉及到未来若干天（x天）的天气预报。

在获取到回复后，它同样从JSON响应中解析出助手的消息，并将其添加到消息列表中。

然后打印出助手的回复消息。

**（通过不同的prompt方式，我们可以让它针对我们告诉它的其他功能。）**

In [11]:
# 初始化一个空的messages列表
messages = []

# 向messages列表添加一条系统角色的消息，要求不做关于函数参数值的假设，如果用户的请求模糊，应该寻求澄清
messages.append({
    "role": "system",  # 消息的角色是"system"
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
})

# 向messages列表添加一条用户角色的消息，用户询问在未来x天内苏格兰格拉斯哥的天气情况
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "what is the weather going to be like in Shanghai, China over the next x days"
})

# 使用定义的chat_completion_request函数发起一个请求，传入messages和functions作为参数
chat_response = chat_completion_request(
    messages, functions=functions
)

# 解析返回的JSON数据，获取助手的回复消息
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的回复消息添加到messages列表中
messages.append(assistant_message)

# 打印助手的回复消息
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: what is the weather going to be like in Shanghai, China over the next x days

assistant[content]: Sure! Could you please provide me with the value of x (the number of days) you would like to know the weather forecast for Shanghai, China?



**(GPT 模型再次要求我们澄清，因为它还没有足够的信息。在这种情况下，它已经知道预测的位置，但需要知道需要多少天的预测。)**

这段代码的主要目标是将用户指定的天数（5天）添加到消息列表中，然后再次调用chat_completion_request函数发起一个请求。

返回的响应中包含了助手对用户的回复，即未来5天的天气预报。

这个预报是基于用户指定的地点（上海）和天数（5天）生成的。

在代码的最后，它解析出返回的JSON响应中的第一个选项，这就是助手的回复消息。

In [12]:
# 向messages列表添加一条用户角色的消息，用户指定接下来的天数为5天
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "5 days"
})

# 使用定义的chat_completion_request函数发起一个请求，传入messages和functions作为参数
chat_response = chat_completion_request(
    messages, functions=functions
)

# 解析返回的JSON数据，获取第一个选项
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的回复消息添加到messages列表中
messages.append(assistant_message)

# 打印助手的回复消息
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: what is the weather going to be like in Shanghai, China over the next x days

assistant[content]: Sure! Could you please provide me with the value of x (the number of days) you would like to know the weather forecast for Shanghai, China?

user: 5 days

assistant[function_call]: {'name': 'get_n_day_weather_forecast', 'arguments': '{"location":"Shanghai, China","format":"celsius","num_days":5}'}



#### 强制使用指定函数

我们可以通过使用`function_call`参数来强制GPT 模型使用指定函数，例如`get_n_day_weather_forecast`。

通过这种方式，可以让 GPT 模型学习如何使用该函数。

In [13]:
# 在这个代码单元中，我们强制GPT 模型使用get_n_day_weather_forecast函数
messages = []  # 创建一个空的消息列表

# 添加系统角色的消息
messages.append({
    "role": "system",  # 角色为系统
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
})

# 添加用户角色的消息
messages.append({
    "role": "user",  # 角色为用户
    "content": "Give me a weather report for San Diego, USA."
})

# 使用定义的chat_completion_request函数发起一个请求，传入messages、functions以及特定的function_call作为参数
chat_response = chat_completion_request(
    messages, functions=functions, function_call={"name": "get_n_day_weather_forecast"}
)

# 解析返回的JSON数据，获取第一个选项
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的回复消息添加到messages列表中
messages.append(assistant_message)

# 打印助手的回复消息
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: Give me a weather report for San Diego, USA.

assistant[function_call]: {'name': 'get_n_day_weather_forecast', 'arguments': '{"location":"San Diego, USA","format":"celsius","num_days":1}'}



下面这段代码演示了在不强制使用特定函数（`get_n_day_weather_forecast`）的情况下，GPT 模型可能会选择不同的方式来回应用户的请求。对于给定的用户请求"Give me a weather report for San Diego, USA."，GPT 模型可能不会调用`get_n_day_weather_forecast`函数。

In [14]:
# 如果我们不强制GPT 模型使用 get_n_day_weather_forecast，它可能不会使用
messages = []  # 创建一个空的消息列表

# 添加系统角色的消息
messages.append({
    "role": "system",  # 角色为系统
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
})

# 添加用户角色的消息
messages.append({
    "role": "user",  # 角色为用户
    "content": "Give me a weather report for San Diego, USA."
})

# 使用定义的chat_completion_request函数发起一个请求，传入messages和functions作为参数
chat_response = chat_completion_request(
    messages, functions=functions
)

# 解析返回的JSON数据，获取第一个选项
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的回复消息添加到messages列表中
messages.append(assistant_message)

# 打印助手的回复消息
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: Give me a weather report for San Diego, USA.

assistant[function_call]: {'name': 'get_current_weather', 'arguments': '{"format":"celsius","location":"San Diego, USA"}'}



#### 强制不使用函数

然后，我们创建另一个消息列表，并添加系统和用户的消息。这次用户请求的是加拿大多伦多当前的天气（使用摄氏度）。

随后，代码再次调用`chat_completion_request`函数，

但这次在`function_call`参数中明确指定了"none"，表示GPT 模型在处理此请求时不能调用任何函数。

最后，代码解析返回的JSON响应，获取第一个选项的消息，即 GPT 模型的回应。

In [16]:
# 创建另一个空的消息列表
messages = []

# 添加系统角色的消息
messages.append({
    "role": "system",  # 角色为系统
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
})

# 添加用户角色的消息
messages.append({
    "role": "user",  # 角色为用户
    "content": "Give me the current weather (use Celcius) for Toronto, Canada."
})

# 使用定义的chat_completion_request函数发起一个请求，传入messages、functions和function_call作为参数
chat_response = chat_completion_request(
    messages, functions=functions, function_call="none"
)

# 解析返回的JSON数据，获取第一个选项
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的回复消息添加到messages列表中
messages.append(assistant_message)

# 打印助手的回复消息
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: Give me the current weather (use Celcius) for Toronto, Canada.

assistant[content]: Sure! Let me get the current weather for Toronto, Canada in Celcius.



## 执行 GPT 模型生成的函数

接着，我们将演示如何执行输入为 GPT 模型生成的函数，并利用这一点来实现一个可以帮助我们回答关于数据库的问题的代理。

为了简单起见，我们将使用[Chinook样本数据库](https://www.sqlitetutorial.net/sqlite-sample-database/)。

![chinook_db](images/chinook_db.jpeg)

*注意：* 在生产环境中，SQL生成可能存在较高风险，因为GPT 模型在生成正确的SQL方面并不完全可靠。

### 定义一个执行SQL查询的函数

首先，让我们定义一些有用的实用函数来从SQLite数据库中提取数据。

In [17]:
import sqlite3

conn = sqlite3.connect("data/chinook.db")
print("Opened database successfully")

Opened database successfully


![chinook](images/chinook.png)

首先定义三个函数`get_table_names`、`get_column_names`和`get_database_info`，用于从数据库连接对象中获取数据库的表名、表的列名以及整体数据库的信息。

In [18]:
def get_table_names(conn):
    """返回一个包含所有表名的列表"""
    table_names = []  # 创建一个空的表名列表
    # 执行SQL查询，获取数据库中所有表的名字
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    # 遍历查询结果，并将每个表名添加到列表中
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names  # 返回表名列表


def get_column_names(conn, table_name):
    """返回一个给定表的所有列名的列表"""
    column_names = []  # 创建一个空的列名列表
    # 执行SQL查询，获取表的所有列的信息
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    # 遍历查询结果，并将每个列名添加到列表中
    for col in columns:
        column_names.append(col[1])
    return column_names  # 返回列名列表


def get_database_info(conn):
    """返回一个字典列表，每个字典包含一个表的名字和列信息"""
    table_dicts = []  # 创建一个空的字典列表
    # 遍历数据库中的所有表
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)  # 获取当前表的所有列名
        # 将表名和列名信息作为一个字典添加到列表中
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts  # 返回字典列表

将数据库信息转换为 Python 字典类型

In [19]:
# 获取数据库信息，并存储为字典列表
database_schema_dict = get_database_info(conn)

# 将数据库信息转换为字符串格式，方便后续使用
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

In [20]:
database_schema_dict

[{'table_name': 'albums', 'column_names': ['AlbumId', 'Title', 'ArtistId']},
 {'table_name': 'sqlite_sequence', 'column_names': ['name', 'seq']},
 {'table_name': 'artists', 'column_names': ['ArtistId', 'Name']},
 {'table_name': 'customers',
  'column_names': ['CustomerId',
   'FirstName',
   'LastName',
   'Company',
   'Address',
   'City',
   'State',
   'Country',
   'PostalCode',
   'Phone',
   'Fax',
   'Email',
   'SupportRepId']},
 {'table_name': 'employees',
  'column_names': ['EmployeeId',
   'LastName',
   'FirstName',
   'Title',
   'ReportsTo',
   'BirthDate',
   'HireDate',
   'Address',
   'City',
   'State',
   'Country',
   'PostalCode',
   'Phone',
   'Fax',
   'Email']},
 {'table_name': 'genres', 'column_names': ['GenreId', 'Name']},
 {'table_name': 'invoices',
  'column_names': ['InvoiceId',
   'CustomerId',
   'InvoiceDate',
   'BillingAddress',
   'BillingCity',
   'BillingState',
   'BillingCountry',
   'BillingPostalCode',
   'Total']},
 {'table_name': 'invoice_i

然后，定义一个函数`ask_database`。

目标是让 GPT 模型帮我们构造一个完整的 SQL 查询。

In [21]:
# 定义一个功能列表，其中包含一个功能字典，该字典定义了一个名为"ask_database"的功能，用于回答用户关于音乐的问题
functions = [
    {
        "name": "ask_database",
        "description": "Use this function to answer user questions about music. Output should be a fully formed SQL query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            SQL query extracting info to answer the user's question.
                            SQL should be written using this database schema:
                            {database_schema_string}
                            The query should be returned in plain text, not in JSON.
                            """,
                }
            },
            "required": ["query"],
        },
    }
]


### 执行 SQL 查询

首先，定义两个函数`ask_database`和`execute_function_call`
- 前者用于实际执行 SQL 查询并返回结果
- 后者用于根据消息中的功能调用信息来执行相应的功能并获取结果

然后，创建一个消息列表，并向其中添加了一个系统消息和一个用户消息。系统消息的内容是指示对话的目标，用户消息的内容是用户的问题。

接着，使用`chat_completion_request`函数发出聊天请求并获取响应，然后从响应中提取出助手的消息并添加到消息列表中。

如果助手的消息中包含功能调用，那么就使用`execute_function_call`函数执行这个功能调用并获取结果，然后将结果作为一个功能消息添加到消息列表中。

最后，使用`pretty_print_conversation`函数打印出整个对话。

In [22]:
def ask_database(conn, query):
    """使用 query 来查询 SQLite 数据库的函数。"""
    try:
        results = str(conn.execute(query).fetchall())  # 执行查询，并将结果转换为字符串
    except Exception as e:  # 如果查询失败，捕获异常并返回错误信息
        results = f"query failed with error: {e}"
    return results  # 返回查询结果


def execute_function_call(message):
    """执行函数调用"""
    # 判断功能调用的名称是否为 "ask_database"
    if message["function_call"]["name"] == "ask_database":
        # 如果是，则获取功能调用的参数，这里是 SQL 查询
        query = json.loads(message["function_call"]["arguments"])["query"]
        # 使用 ask_database 函数执行查询，并获取结果
        results = ask_database(conn, query)
    else:
        # 如果功能调用的名称不是 "ask_database"，则返回错误信息
        results = f"Error: function {message['function_call']['name']} does not exist"
    return results  # 返回结果

In [23]:
# 创建一个空的消息列表
messages = []

# 向消息列表中添加一个系统角色的消息，内容是 "Answer user questions by generating SQL queries against the Chinook Music Database."
messages.append({"role": "system", "content": "Answer user questions by generating SQL queries against the Chinook Music Database."})

# 向消息列表中添加一个用户角色的消息，内容是 "Hi, who are the top 5 artists by number of tracks?"
messages.append({"role": "user", "content": "Hi, who are the top 5 artists by number of tracks?"})

# 使用 chat_completion_request 函数获取聊天响应
chat_response = chat_completion_request(messages, functions)

# 从聊天响应中获取助手的消息
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的消息添加到消息列表中
messages.append(assistant_message)

# 如果助手的消息中有功能调用
if assistant_message.get("function_call"):
    # 使用 execute_function_call 函数执行功能调用，并获取结果
    results = execute_function_call(assistant_message)
    # 将功能的结果作为一个功能角色的消息添加到消息列表中
    messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": results})

# 使用 pretty_print_conversation 函数打印对话
pretty_print_conversation(messages)


system: Answer user questions by generating SQL queries against the Chinook Music Database.

user: Hi, who are the top 5 artists by number of tracks?

assistant[function_call]: {'name': 'ask_database', 'arguments': '{"query":"SELECT artists.Name AS Artist, COUNT(tracks.TrackId) AS TrackCount\\nFROM artists\\nJOIN albums ON artists.ArtistId = albums.ArtistId\\nJOIN tracks ON albums.AlbumId = tracks.AlbumId\\nGROUP BY Artist\\nORDER BY TrackCount DESC\\nLIMIT 5;"}'}

function (ask_database): [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]



In [24]:
database_schema_dict

[{'table_name': 'albums', 'column_names': ['AlbumId', 'Title', 'ArtistId']},
 {'table_name': 'sqlite_sequence', 'column_names': ['name', 'seq']},
 {'table_name': 'artists', 'column_names': ['ArtistId', 'Name']},
 {'table_name': 'customers',
  'column_names': ['CustomerId',
   'FirstName',
   'LastName',
   'Company',
   'Address',
   'City',
   'State',
   'Country',
   'PostalCode',
   'Phone',
   'Fax',
   'Email',
   'SupportRepId']},
 {'table_name': 'employees',
  'column_names': ['EmployeeId',
   'LastName',
   'FirstName',
   'Title',
   'ReportsTo',
   'BirthDate',
   'HireDate',
   'Address',
   'City',
   'State',
   'Country',
   'PostalCode',
   'Phone',
   'Fax',
   'Email']},
 {'table_name': 'genres', 'column_names': ['GenreId', 'Name']},
 {'table_name': 'invoices',
  'column_names': ['InvoiceId',
   'CustomerId',
   'InvoiceDate',
   'BillingAddress',
   'BillingCity',
   'BillingState',
   'BillingCountry',
   'BillingPostalCode',
   'Total']},
 {'table_name': 'invoice_i

In [25]:
# 向消息列表中添加一个用户的问题，内容是 "What is the name of the album with the most tracks?"
messages.append({"role": "user", "content": "What is the name of the album with the most tracks?"})

# 使用 chat_completion_request 函数获取聊天响应
chat_response = chat_completion_request(messages, functions)

# 从聊天响应中获取助手的消息
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的消息添加到消息列表中
messages.append(assistant_message)

# 如果助手的消息中有功能调用
if assistant_message.get("function_call"):
    # 使用 execute_function_call 函数执行功能调用，并获取结果
    results = execute_function_call(assistant_message)
    # 将功能的结果作为一个功能角色的消息添加到消息列表中
    messages.append({"role": "function", "content": results, "name": assistant_message["function_call"]["name"]})

# 使用 pretty_print_conversation 函数打印对话
pretty_print_conversation(messages)

system: Answer user questions by generating SQL queries against the Chinook Music Database.

user: Hi, who are the top 5 artists by number of tracks?

assistant[function_call]: {'name': 'ask_database', 'arguments': '{"query":"SELECT artists.Name AS Artist, COUNT(tracks.TrackId) AS TrackCount\\nFROM artists\\nJOIN albums ON artists.ArtistId = albums.ArtistId\\nJOIN tracks ON albums.AlbumId = tracks.AlbumId\\nGROUP BY Artist\\nORDER BY TrackCount DESC\\nLIMIT 5;"}'}

function (ask_database): [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]

user: What is the name of the album with the most tracks?

assistant[function_call]: {'name': 'ask_database', 'arguments': '{"query":"SELECT albums.Title AS Album, COUNT(tracks.TrackId) AS TrackCount\\nFROM albums\\nJOIN tracks ON albums.AlbumId = tracks.AlbumId\\nGROUP BY Album\\nORDER BY TrackCount DESC\\nLIMIT 1;"}'}

function (ask_database): [('Greatest Hits', 57)]



In [30]:
# messages.append({"role": "user", "content": "最受欢迎的前10个专辑类型是什么?"})
# messages.append({"role": "user", "content": "最受欢迎的前10个歌手是谁?受欢迎是指销售数量最多"})
messages.append({"role": "user", "content": "创造最高收入的前10个歌手是谁？"})

chat_response = chat_completion_request(messages, functions)
assistant_message = chat_response.json()["choices"][0]["message"]

messages.append(assistant_message)

if assistant_message.get("function_call"):
    results = execute_function_call(assistant_message)
    messages.append({"role": "function", "content": results, "name": assistant_message["function_call"]["name"]})

pretty_print_conversation(messages)

system: Answer user questions by generating SQL queries against the Chinook Music Database.

user: Hi, who are the top 5 artists by number of tracks?

assistant[function_call]: {'name': 'ask_database', 'arguments': '{"query":"SELECT artists.Name AS Artist, COUNT(tracks.TrackId) AS TrackCount\\nFROM artists\\nJOIN albums ON artists.ArtistId = albums.ArtistId\\nJOIN tracks ON albums.AlbumId = tracks.AlbumId\\nGROUP BY Artist\\nORDER BY TrackCount DESC\\nLIMIT 5;"}'}

function (ask_database): [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]

user: What is the name of the album with the most tracks?

assistant[function_call]: {'name': 'ask_database', 'arguments': '{"query":"SELECT albums.Title AS Album, COUNT(tracks.TrackId) AS TrackCount\\nFROM albums\\nJOIN tracks ON albums.AlbumId = tracks.AlbumId\\nGROUP BY Album\\nORDER BY TrackCount DESC\\nLIMIT 1;"}'}

function (ask_database): [('Greatest Hits', 57)]

user: 最受欢迎的专辑类型是什么?

assistant[functio

## Homework 1

**使用第三方天气查询API，实现完整可执行的天气查询应用**

In [45]:
# get current weather is a function to use free api to get current weather for a specific location and format (celsius or felheight) for today
def get_current_weather(location: str, format: str = "celsius"):
    """Get the current weather for a location"""
    import requests
    
    # Replace this with your actual API key from OpenWeatherMap
    API_KEY = "ee667a81a67b46838b9345f656644f81"  
    
    BASE_URL = "http://api.openweathermap.org/data/2.5/weather"
    
    params = {
        'q': location,
        'appid': API_KEY,
        'units': 'metric' if format == "celsius" else "imperial"
    }
    
    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        
        weather_data = response.json()
        temp = weather_data['main']['temp']
        conditions = weather_data['weather'][0]['description']
        humidity = weather_data['main']['humidity']
        
        temp_unit = "°C" if format == "celsius" else "°F"
        
        weather_info = (
            f"Current weather in {location}:\n"
            f"Temperature: {temp}{temp_unit}\n"
            f"Conditions: {conditions}\n"
            f"Humidity: {humidity}%"
        )
        
        return weather_info
        
    except requests.exceptions.RequestException as e:
        return f"Error fetching weather data: {str(e)}"
    


In [84]:
def _get_weather_base_config():
    """Return base configuration for weather API calls"""
    return {
        'api_key': "ee667a81a67b46838b9345f656644f81",
        'base_url': "http://api.openweathermap.org/data/2.5",
        'units_map': {
            'celsius': 'metric',
            'fahrenheit': 'imperial'
        }
    }

def _make_weather_request(endpoint: str, params: dict) -> dict:
    """Make a weather API request and handle common error cases"""
    import requests
    
    config = _get_weather_base_config()
    url = f"{config['base_url']}/{endpoint}"
    params['appid'] = config['api_key']
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        raise Exception(f"Error fetching weather data: {str(e)}")

def get_current_weather(location: str, format: str = "celsius") -> str:
    """Get the current weather for a location"""
    config = _get_weather_base_config()
    
    params = {
        'q': location,
        'units': config['units_map'][format]
    }
    
    try:
        weather_data = _make_weather_request('weather', params)
        temp = weather_data['main']['temp']
        conditions = weather_data['weather'][0]['description']
        humidity = weather_data['main']['humidity']
        
        temp_unit = "°C" if format == "celsius" else "°F"
        
        return (
            f"Current weather in {location}:\n"
            f"Temperature: {temp}{temp_unit}\n"
            f"Conditions: {conditions}\n"
            f"Humidity: {humidity}%"
        )
    except Exception as e:
        return str(e)

def get_n_day_weather_forecast(location: str, format: str = "celsius", num_days: int = 5) -> str:
    """Get an N-day weather forecast for a location"""
    config = _get_weather_base_config()
    
    # OpenWeather API forecast endpoint returns data points every 3 hours
    # To get N days worth of data, we need N * 8 data points
    params = {
        'q': location,
        'units': config['units_map'][format],
        'cnt': min(num_days * 8, 40)  # API limits to 40 data points (5 days)
    }
    
    try:
        forecast_data = _make_weather_request('forecast', params)
        temp_unit = "°C" if format == "celsius" else "°F"
        
        # Group forecasts by day
        daily_forecasts = {}
        for item in forecast_data['list']:
            from datetime import datetime
            date = datetime.fromtimestamp(item['dt']).strftime('%Y-%m-%d')
            if date not in daily_forecasts:
                daily_forecasts[date] = {
                    'temps': [],
                    'conditions': [],
                    'dt': item['dt']  # Store timestamp to properly sort by date
                }
            daily_forecasts[date]['temps'].append(item['main']['temp'])
            daily_forecasts[date]['conditions'].append(item['weather'][0]['description'])
        
        # Sort forecasts by date
        sorted_forecasts = sorted(daily_forecasts.items(), key=lambda x: x[1]['dt'])
        
        # Format the response
        forecast_text = f"\n{num_days}-day forecast for {location}:\n"
        for date, data in sorted_forecasts[:num_days]:
            avg_temp = sum(data['temps']) / len(data['temps'])
            main_condition = max(set(data['conditions']), key=data['conditions'].count)
            forecast_text += (
                f"\n{date}:\n"
                f"Average Temperature: {avg_temp:.1f}{temp_unit}\n"
                f"Conditions: {main_condition}\n"
            )
        
        return forecast_text
        
    except Exception as e:
        return str(e)

In [85]:
get_current_weather("hangzhou, china", "celsius")

'Current weather in hangzhou, china:\nTemperature: 11.56°C\nConditions: clear sky\nHumidity: 29%'

In [86]:
get_n_day_weather_forecast("hangzhou, china", "celsius", 5)

'\n5-day forecast for hangzhou, china:\n\n2025-01-06:\nAverage Temperature: 9.5°C\nConditions: broken clouds\n\n2025-01-07:\nAverage Temperature: 6.8°C\nConditions: overcast clouds\n\n2025-01-08:\nAverage Temperature: 8.0°C\nConditions: overcast clouds\n\n2025-01-09:\nAverage Temperature: 3.5°C\nConditions: few clouds\n\n2025-01-10:\nAverage Temperature: 3.3°C\nConditions: clear sky\n'

In [87]:
def execute_weather_function_call(message):
    """执行函数调用"""
    if message["function_call"]["name"] == "get_current_weather":
        args = json.loads(message["function_call"]["arguments"])
        location = args["location"]
        format = args["format"]
        results = get_current_weather(location=location, format=format)
    elif message["function_call"]["name"] == "get_n_day_weather_forecast":
        args = json.loads(message["function_call"]["arguments"])
        location = args["location"]
        format = args["format"]
        num_days = args["num_days"]
        results = get_n_day_weather_forecast(location=location, format=format, num_days=num_days)
    else:
        results = f"Error: function {message['function_call']['name']} does not exist"
    return results  # 返回结果

In [91]:
messages = []

messages.append({
    "role": "system",  # 消息的角色是"system"
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
})

messages.append({
    "role": "user",  # 消息的角色是"user"
    # "content": "what is the weather going to be like for today?"
    "content": "今天天气怎么样？"
})

chat_response = chat_completion_request(
    messages, functions=functions
)

assistant_message = chat_response.json()["choices"][0]["message"]

if assistant_message.get("function_call"):
    results = execute_weather_function_call(assistant_message)
    # print(assistant_message)
    # print(functions)
    messages.append({"role": "function", "content": results, "name": assistant_message["function_call"]["name"]})
else:
    messages.append(assistant_message)
# messages.append(assistant_message)

pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: 今天天气怎么样？

assistant[content]: 请问您在哪个城市或地区？



In [92]:
messages.append({
    "role": "user",
    # "content": "hangzhou, china"
    "content": "中国杭州"
})

chat_response = chat_completion_request(
    messages, functions=functions
)

assistant_message = chat_response.json()["choices"][0]["message"]

if assistant_message.get("function_call"):
    results = execute_weather_function_call(assistant_message)
    # print(assistant_message)
    # print(functions)
    messages.append({"role": "function", "content": results, "name": assistant_message["function_call"]["name"]})
else:
    messages.append(assistant_message)

pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: 今天天气怎么样？

assistant[content]: 请问您在哪个城市或地区？

user: 中国杭州

function (get_current_weather): Current weather in Hangzhou, China:
Temperature: 10.95°C
Conditions: few clouds
Humidity: 35%



In [93]:
messages.append({
    "role": "user",
    # "content": "what's the weather like in hangzhou, china for next 7 days?"
    "content": "给我杭州未来7天的天气情况"
})

chat_response = chat_completion_request(
    messages, functions=functions
)

assistant_message = chat_response.json()["choices"][0]["message"]

if assistant_message.get("function_call"):
    results = execute_weather_function_call(assistant_message)
    messages.append({"role": "function", "content": results, "name": assistant_message["function_call"]["name"]})
else:
    messages.append(assistant_message)

pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: 今天天气怎么样？

assistant[content]: 请问您在哪个城市或地区？

user: 中国杭州

function (get_current_weather): Current weather in Hangzhou, China:
Temperature: 10.95°C
Conditions: few clouds
Humidity: 35%

user: 给我杭州未来7天的天气情况

assistant[content]: 请问您希望使用摄氏度还是华氏度来查看未来7天的天气情况？



In [94]:
messages.append({
    "role": "user",
    # "content": "what's the weather like in hangzhou, china for next 7 days?"
    "content": "摄氏度"
})

chat_response = chat_completion_request(
    messages, functions=functions
)

assistant_message = chat_response.json()["choices"][0]["message"]

if assistant_message.get("function_call"):
    results = execute_weather_function_call(assistant_message)
    messages.append({"role": "function", "content": results, "name": assistant_message["function_call"]["name"]})
else:
    messages.append(assistant_message)

pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: 今天天气怎么样？

assistant[content]: 请问您在哪个城市或地区？

user: 中国杭州

function (get_current_weather): Current weather in Hangzhou, China:
Temperature: 10.95°C
Conditions: few clouds
Humidity: 35%

user: 给我杭州未来7天的天气情况

assistant[content]: 请问您希望使用摄氏度还是华氏度来查看未来7天的天气情况？

user: 摄氏度

function (get_n_day_weather_forecast): 
7-day forecast for Hangzhou, China:

2025-01-06:
Average Temperature: 9.5°C
Conditions: broken clouds

2025-01-07:
Average Temperature: 6.8°C
Conditions: overcast clouds

2025-01-08:
Average Temperature: 8.0°C
Conditions: overcast clouds

2025-01-09:
Average Temperature: 3.5°C
Conditions: few clouds

2025-01-10:
Average Temperature: 3.3°C
Conditions: clear sky

2025-01-11:
Average Temperature: 4.1°C
Conditions: overcast clouds


